# 利用Qwen 翻译题目

In [11]:
import os
import time

import pandas as pd
from openai import OpenAI
import pandas

In [12]:
language_list = ["c&cpp"]
suffix_map = {
    "python": "py",
    "javascript": "js",
    "typescript": "ts",
    "java": "java",
    "c&cpp": "cpp"
}

import httpx

client = OpenAI(
    base_url="https://api.xty.app/v1",
    api_key="sk-FXsYo3kFBrjLGBCLBdAf99Cf83264aCd8dFdDdCc0e73Aa0c",
    http_client=httpx.Client(
        base_url="https://api.xty.app/v1",
        follow_redirects=True,
    ),
)
model_name = "gpt-3.5-turbo"

In [13]:
prompt_message_template = {
    # py 2 js
    "python_answer_to_javascript": "You are a highly skilled developer and programming expert proficient in both Python and JavaScript. Your task is to convert the following Python code into its equivalent JavaScript code, ensuring that the functionality, logic, and structure remain consistent. Here is the Python code: %s",
    "python_signature_to_javascript": "You are an expert in both Python and JavaScript programming. Your task is to convert Python function signatures or class signatures into equivalent JavaScript syntax, while preserving the purpose and structure of the code. Additionally, you must ensure that Python comments are rewritten to conform to JavaScript documentation standards using JSDoc format. Here is the Python code with its function/class signature and comments: %s",
    "python_test_to_javascript": "You are an expert in Python's unittest framework and JavaScript's jest testing framework. Your task is to convert the given Python unittest test class into an equivalent test class using jest. Make sure that the functionality and logic of the tests are maintained. Any necessary changes due to language differences between Python and JavaScript should be documented in the conversion. Additionally, ensure that the structure is clear and that all assertions in unittest are mapped to appropriate jest equivalents. Here is the Python code: %s",
    # py 2 ts
    "python_answer_to_typescript": "You are an expert in Python and TypeScript programming. Your task is to convert the provided Python code into equivalent TypeScript code. Ensure that the functionality, structure, and logic remain consistent, while also adjusting for any necessary language differences. You should properly define types in TypeScript, following TypeScript's static type system. Additionally, any Python-specific features should be adapted to idiomatic TypeScript where needed. Here is the Python code: %s",
    "python_signature_to_typescript": "You are proficient in both Python and TypeScript programming. Your task is to convert Python function signatures or class signatures into equivalent TypeScript code, while preserving the original functionality and structure. In addition, convert Python comments and docstrings to appropriate TypeScript documentation comments using the /** */ JSDoc style. Ensure that type annotations in Python are properly translated to TypeScript types, and make necessary adjustments for any language differences. Here is the Python code: %s",
    "python_test_to_typescript": "You are proficient in both Python's unittest framework and TypeScript's jest testing framework. Your task is to convert the provided Python unittest test class into an equivalent test class using jest in TypeScript. Make sure that the functionality and logic of the tests are preserved. Adjust the code to account for the differences between Python and TypeScript, and ensure that the necessary type annotations are included in the TypeScript code. The assert methods in unittest should be mapped to the appropriate jest assertions. Here is the Python code: %s",
    # py 2 c&cpp
    "python_answer_to_c&cpp": "You are an expert in both Python and C++ programming. Your task is to convert the provided Python code into equivalent C++ code. Ensure that the functionality, logic, and structure remain consistent while adapting to the syntax and features of C++. Make sure to handle any language-specific features, such as type declarations, memory management, or loops, and translate them into their appropriate C++ counterparts. Here is the Python code: %s",
    "python_signature_to_c&cpp": "You are proficient in both Python and C++ programming. Your task is to convert Python function signatures or class signatures into their equivalent C++ versions, while preserving the original purpose and structure of the code. You must also convert Python comments and docstrings into appropriate C++ comments using // for single-line comments and /* */ for multi-line comments, adhering to C++ documentation standards. Additionally, ensure that Python type annotations are correctly translated into C++ types. Here is the Python code: %s",
    "python_test_to_c&cpp": "You are proficient in both Python's unittest framework and C++'s Catch2 testing framework. Your task is to convert the provided Python unittest test class into an equivalent test class using Catch2 in C++. Ensure that the functionality and logic of the tests are preserved. Make any necessary adjustments for language differences between Python and C++, and use appropriate assertions in Catch2. Here is the Python code: %s",
    # py 2 java
    "python_answer_to_java": "You are an expert in both Python and Java programming. Your task is to convert the provided Python code into equivalent Java code. The Java code should be organized with the package name org.real.temp and the class name Answer. Ensure that the functionality, logic, and structure remain consistent while adapting to Java's syntax and language features. Make sure to handle type declarations, method signatures, and object-oriented principles in Java appropriately. Here is the Python code:  %s",
    "python_signature_to_java": "You are proficient in both Python and Java programming. Your task is to convert Python function signatures or class signatures into their equivalent Java versions, while preserving the original functionality and structure. Additionally, you must convert Python comments and docstrings into appropriate Java-style comments using the /** */ Javadoc format. Make sure that Python type annotations are translated into appropriate Java types, and that the method or class is structured according to Java's conventions. Here is the Python code: %s",
    "python_test_to_java": "You are proficient in both Python's unittest framework and Java's JUnit testing framework. Your task is to convert the provided Python unittest test class into an equivalent test class using JUnit in Java. The Java test class should be placed in the package org.real.temp and named Tester. Ensure that the functionality and logic of the tests are preserved, and use appropriate JUnit annotations (e.g., @Test) and assertion methods. Make necessary adjustments for differences between Python and Java syntax and structure. Here is the Python code: %s",

    # js 2 py
    "javascript_answer_to_python": "You are an expert in both JavaScript and Python programming. Your task is to convert the provided JavaScript code into equivalent Python code. Ensure that the functionality, logic, and structure remain consistent, while adapting the syntax and language-specific features of JavaScript to their appropriate Python counterparts. Handle any necessary changes in variable declaration, loops, and functions, and ensure the Python version maintains the same behavior as the original JavaScript code. Here is the JavaScript code: %s",
    "javascript_signature_to_python": "You are proficient in both JavaScript and Python programming. Your task is to convert JavaScript function signatures or class signatures into equivalent Python versions, while preserving the original functionality and structure. Additionally, you must convert JavaScript comments (both single-line // and multi-line /* */) into appropriate Python-style comments (using # for single-line comments and \"\"\" for multi-line docstrings). Make sure to adapt JavaScript-specific syntax and conventions to Pythonic equivalents. Here is the JavaScript code: %s",
    "javascript_test_to_python": "please change this javascript jest testcase to python testcase use unittest, the javascript jest testcase code as below %s",
    # js 2 ts
    "javascript_answer_to_typescript": "please change this javascript code to typescript code,the javascript code as below %s",
    "javascript_signature_to_typescript": "please change this javascript code signature to typescript code signature,keep the code comments, the javascript code signature as below %s",
    "javascript_test_to_typescript": "please change this javascript jest testcase to typescript testcase use jest describe testcase, the javascript jest testcase code as below %s",
    # js 2 c&cpp
    "javascript_answer_to_c&cpp": "please change this javascript code to cpp code,the javascript code as below %s",
    "javascript_signature_to_c&cpp": "please change this javascript code signature to cpp code signature,keep the code comments, the javascript code signature as below %s",
    "javascript_test_to_c&cpp": "please change this javascript jest testcase to cpp testcase use catch2 testcase,the javascript jest testcase code as below %s",
    # js 2 java
    "javascript_answer_to_java": "please change this javascript code to java code,the javascript code as below %s",
    "javascript_signature_to_java": "please change this javascript code signature to java code signature,keep the code comments, the javascript code signature as below %s",
    "javascript_test_to_java": "please change this javascript jest testcase to java testcase use junit 4,package is org.real.temp,class is Tester,the javascript jest testcode as below %s",

    # ts 2 py
    "typescript_answer_to_python": "please change this typescript code to python code,the typescript code as below %s",
    "typescript_signature_to_python": "please change this typescript code signature to python code signature,keep the code comments, the typescript code signature as below %s",
    "typescript_test_to_python": "please change this typescript jest testcase to python testcase use unittest,the typescript jest testcase code as below %s",

    # ts 2 js
    "typescript_answer_to_javascript": "please change this typescript code to javascript code,the typescript code as below %s",
    "typescript_signature_to_javascript": "please change this typescript code signature to javascript code signature,keep the code comments, the typescript code signature as below %s",
    "typescript_test_to_javascript": "please change this typescript jest testcase to javascript testcase use jest describe testcase, the typescript jest testcase code as below %s",

    # ts 2 c&cpp
    "typescript_answer_to_c&cpp": "please change this typescript code to cpp code,the typescript code as below %s",
    "typescript_signature_to_c&cpp": "please change this typescript code signature to cpp code signature,keep the code comments, the typescript code signature as below %s",
    "typescript_test_to_c&cpp": "please change this typescript jest testcase to cpp testcase use catch2 testcase,the typescript jest testcase code as below %s",

    # ts 2 java
    "typescript_answer_to_java": "please change this typescript code to java code,the typescript code as below %s",
    "typescript_signature_to_java": "please change this typescript code signature to java code signature,keep the code comments, the typescript code signature as below %s",
    "typescript_test_to_java": "please change this typescript jest testcase to java testcase use junit,package is org.real.temp,class is Tester,the javascript jest testcode as below %s",

    # cpp 2 python
    "cpp_answer_to_python": "You are an expert programmer who can convert C++ code into Python code.    Given the C++ code below, translate it into Python, ensuring that the logic and functionality remain the same, and adapting any syntax differences.    Please use Pythonic conventions and best practices where appropriate.C++ code: %s",
    "cpp_signature_to_python": "You are proficient in both C++ and Python programming.  Your task is to convert C++ function signatures or class signatures into their equivalent Python versions while preserving the original functionality and structure.  Additionally, convert C++ comments (// for single-line and /* */ for multi-line comments) into appropriate Python-style comments using # for single-line comments and \"\"\" for multi-line docstrings.  Make sure that the C++ type declarations and memory management conventions are adapted to Pythonic equivalents.  Here is the C++ code: %s",
    "cpp_test_to_python": "You are proficient in both C++'s Catch2 framework and Python's unittest framework.  Your task is to convert the provided C++ Catch2 test cases into equivalent test cases using Python's unittest framework.  Ensure that the logic and functionality of the tests remain consistent.  Make necessary adjustments for the language and framework differences, including importing the appropriate unittest modules in Python and using the correct assertions.  Here is the C++ code: %s",

    # c&cpp 2 javascript
    "c&cpp_answer_to_javascript": "You are an expert programmer who can convert C++ code into JavaScript code.  Given the C++ code below, translate it into JavaScript, ensuring that the logic and functionality remain the same, and adapting any syntax differences.    Please use JavaScript conventions and best practices where appropriate.C++ code: %s",
    "c&cpp_signature_to_javascript": "You are proficient in both C++ and JavaScript programming.  Your task is to convert C++ function signatures or class signatures into their equivalent JavaScript versions while preserving the original functionality and structure.  Additionally, convert C++ comments (// for single-line and /* */ for multi-line comments) into appropriate JavaScript-style comments.Make sure that the C++ type declarations and memory management conventions are adapted to JavaScript equivalents.  Here is the C++ code: %s",
    "c&cpp_test_to_javascript": "You are proficient in both C++'s Catch2 framework and JavaScript's jest framework.  Your task is to convert the provided C++ Catch2 test cases into equivalent test cases using JavaScript's jest framework.  Ensure that the logic and functionality of the tests remain consistent.  Make necessary adjustments for the language and framework differences, using the correct assertions.  Here is the C++ code: %s",
    # c&cpp 2 typescript
    "c&cpp_answer_to_typescript": "You are an expert programmer who can convert C++ code into TypeScript code.  Given the C++ code below, translate it into TypeScript, ensuring that the logic and functionality remain the same, and adapting any syntax differences. Please use TypeScript conventions and best practices where appropriate.C++ code: %s",
    "c&cpp_signature_to_typescript": "You are proficient in both C++ and TypeScript programming.  Your task is to convert C++ function signatures or class signatures into their equivalent TypeScript versions while preserving the original functionality and structure.  Additionally, convert C++ comments (// for single-line and /* */ for multi-line comments) into appropriate TypeScript-style comments.Make sure that the C++ type declarations and memory management conventions are adapted to TypeScript equivalents.  Here is the C++ code: %s",
    "c&cpp_test_to_typescript": "You are proficient in both C++'s Catch2 framework and TypeScript's jest framework.  Your task is to convert the provided C++ Catch2 test cases into equivalent test cases using TypeScript's jest framework.  Ensure that the logic and functionality of the tests remain consistent.  Make necessary adjustments for the language and framework differences, using the correct assertions.  Here is the C++ code: %s",
    # c&cpp 2 java
    "c&cpp_answer_to_java": "You are an expert in both C++ and Java programming. Your task is to convert the provided C++ code into equivalent Java code. The Java code should be organized with the package name org.real.temp and the class name Answer. Ensure that the functionality, logic, and structure remain consistent while adapting to Java's syntax and language features. Make sure to handle type declarations, method signatures, and object-oriented principles in Java appropriately. Here is the C++ code:  %s",
    "c&cpp_signature_to_java": "You are proficient in both C++ and Java programming. Your task is to convert C++ function signatures or class signatures into their equivalent Java versions, while preserving the original functionality and structure. Additionally, you must convert C++ comments and docstrings into appropriate Java-style comments using the /** */ Javadoc format. Make sure that C++ type annotations are translated into appropriate Java types, and that the method or class is structured according to Java's conventions. Here is the C++ code: %s",
    "c&cpp_test_to_java": "You are proficient in both C++'s catch2 framework and Java's JUnit testing framework. Your task is to convert the provided C++ unittest test class into an equivalent test class using JUnit4 in Java. The Java test class should be placed in the package org.real.temp and named Tester. Ensure that the functionality and logic of the tests are preserved, and use appropriate JUnit annotations (e.g., @Test) and assertion methods. Make necessary adjustments for differences between C++ and Java syntax and structure. Here is the C++ code: %s",

    # java 2 python
    "java_answer_to_python": "You are proficient in both Java and Python programming.  Your task is to convert the provided Java function code into its equivalent Python function.  Ensure that the functionality and logic of the code remain consistent while adapting to Python's syntax.  You should only convert the function itself, without converting the entire Java class.  Here is the Java code: %s",
    "java_signature_to_python": "You are proficient in both Java and Python programming.  Your task is to convert Java function signatures into their equivalent Python function signatures.  Ensure that the parameters and return types are properly adapted to Python's syntax.  You only need to convert the function signature, not the entire body of the function.  Here is the Java function signature: %s",
    "java_test_to_python": "You are proficient in both Java's JUnit framework and Python's unittest framework.  Your task is to convert the provided Java JUnit test case into an equivalent test case using Python's unittest framework.  Ensure that the logic and functionality of the tests are preserved while adapting to Python's syntax and conventions.  Make necessary adjustments for the differences between Java and Python, including imports, annotations, and assertions.  Here is the Java code:%s"
}

In [14]:
from pathlib import Path

for language in language_list:
    excel_data = pd.read_excel("./xlsx/RealisticEval-Data.xlsx", sheet_name=language)
    data = excel_data[excel_data['check'] == 'yes']
    l_list = [x for x in ["javascript", "typescript", "c&cpp", "java"] if x != language]
    for index, row in data.iterrows():
        task_id = row['task_id']
        # if task_id in [i for i in range(1, 524)]:
        #     continue
        work_path = f"../all/t{task_id}"
        # 读取signature代码
        signature_path = f"{work_path}/{language}/signature.{suffix_map[language]}"
        with open(signature_path, "r", encoding="utf8") as file:
            signature_code = file.read()
            print(f"signature_code================================\n{signature_code}")
        # 读取test代码
        test_path = f"{work_path}/{language}/test.{suffix_map[language]}"
        with open(test_path, "r", encoding="utf8") as file:
            test_code = file.read()
            print(f"test_code================================\n{test_code}")
        # 读取answer代码
        answer_path = f"{work_path}/{language}/answer.{suffix_map[language]}"
        with open(answer_path, "r", encoding="utf8") as file:
            answer_code = file.read()
            print(f"answer_code================================\n{answer_code}")
        for l in l_list:
            # 创建文件夹
            if l != "java":
                Path(f"{work_path}/{l}").mkdir(parents=True, exist_ok=True)
                answer_file_path = Path(f"{work_path}/{l}/answer.{suffix_map[l]}")
                signature_file_path = Path(f"{work_path}/{l}/signature.{suffix_map[l]}")
                test_file_path = Path(f"{work_path}/{l}/test.{suffix_map[l]}")
                if not answer_file_path.exists():
                    answer_file_path.touch()
                if not signature_file_path.exists():
                    signature_file_path.touch()
                if not test_file_path.exists():
                    test_file_path.touch()
            elif l == "java":
                Path(f"{work_path}/{l}").mkdir(parents=True, exist_ok=True)
                answer_file_path = Path(f"{work_path}/{l}/Answer.{suffix_map[l]}")
                signature_file_path = Path(f"{work_path}/{l}/Signature.{suffix_map[l]}")
                test_file_path = Path(f"{work_path}/{l}/Tester.{suffix_map[l]}")
                if not answer_file_path.exists():
                    answer_file_path.touch()
                if not signature_file_path.exists():
                    signature_file_path.touch()
                if not test_file_path.exists():
                    test_file_path.touch()
            print(f"{language}_{task_id}_to_{l} ===============================\n")

            # 答案转换 =============================
            message_list = []
            answer_key = f"{language}_answer_to_{l}"
            answer_template = prompt_message_template[answer_key] % (answer_code)
            message_list.append({'role': 'user', 'content': answer_template})
            while True:
                try:
                    completion = client.chat.completions.create(
                        model=model_name,  # 模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
                        messages=message_list,
                        temperature=0.1
                    )
                    llm_answer_response = completion.choices[0].message.content
                    print(llm_answer_response)
                    message_list.append({'role': 'assistant', 'content': llm_answer_response})
                    with open(f"{work_path}/{l}/answer.txt", "w", encoding="utf8") as file:
                        file.write(llm_answer_response)
                        file.flush()
                    break
                except Exception as e:
                    print(e)
                    continue

            # 签名转换=============================
            signature_key = f"{language}_signature_to_{l}"
            signature_template = prompt_message_template[signature_key] % (signature_code)
            message_list.append({'role': 'user', 'content': signature_template})
            while True:
                try:
                    completion = client.chat.completions.create(
                        model=model_name,  # 模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
                        messages=message_list,
                        temperature=0.1
                    )
                    llm_signature_response = completion.choices[0].message.content
                    print(llm_signature_response)
                    message_list.append({'role': 'assistant', 'content': llm_signature_response})
                    with open(f"{work_path}/{l}/signature.txt", "w", encoding="utf8") as file:
                        file.write(llm_signature_response)
                        file.flush()
                    break
                except Exception as e:
                    print(e)
                    continue

            # 测试转换=============================
            test_key = f"{language}_test_to_{l}"
            test_template = prompt_message_template[test_key] % (test_code)
            message_list.append({'role': 'user', 'content': test_template})
            while True:
                try:
                    completion = client.chat.completions.create(
                        model=model_name,  # 模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
                        messages=message_list,
                        temperature=0.1
                    )
                    llm_test_response = completion.choices[0].message.content
                    print(llm_test_response)
                    with open(f"{work_path}/{l}/test.txt", "w", encoding="utf8") as file:
                        file.write(llm_test_response)
                        file.flush()
                    break
                except Exception as e:
                    print(e)
                    continue
            time.sleep(0.5)

signature_code================================
#include <iostream>
#include <cmath>

struct Point {
    float x, y;
};

struct Ray {
    Point origin;   // Starting point of the ray
    Point direction; // Direction of the ray (should be normalized)
};

struct Circle {
    Point center; // Center of the circle
    float radius; // Radius of the circle
};

/**
 * @brief Determines whether a ray intersects with a circle.
 *
 * This function checks if a given ray intersects with a specified circle
 * in a 2D space. It performs mathematical calculations to determine if
 * the ray, defined by its origin and direction, crosses the area of the
 * circle defined by its center and radius.
 *
 * @param ray The ray to be tested for intersection. It is assumed to
 * contain properties such as an origin point and a direction vector.
 * @param circle The circle to check for intersection. It is assumed to
 * contain properties such as a center point and a radius.
 *
 * @return true if the ray interse